### Clone keras yolov3 repository

In [1]:
%cd /data1
!git clone https://github.com/qqwweee/keras-yolo3.git keras_yolo3

/data1
fatal: destination path 'keras_yolo3' already exists and is not an empty directory.


### Load and convert yolov3 model to keras

In [ ]:
%cd keras_yolo3
!wget https://pjreddie.com/media/files/yolov3.weights
!python convert.py yolov3.cfg yolov3.weights /data1/yolo.h5

/data1/keras_yolo3
--2021-08-24 08:12:35--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights.1’

yolov3.weights.1     59%[==========>         ] 141.12M  7.76MB/s    eta 13s    

### Download kneron images for further quantization

In [ ]:
%cd /data1
!wget http://doc.kneron.com/docs/toolchain/res/test_image10.zip
!unzip test_image10.zip

### Copy image for model's testing in different convertation stages

In [ ]:
%cd /data1
!cp /workspace/E2E_Simulator/app/test_image_folder/yolo/000000350003.jpg ./.

### Run example script test.py to convert yolov3 from keras to nef

In [ ]:
!python /workspace/Kneron_yolov3_inference/test.py

### Make changes to E2E simulator in order to use it with multiprocessing

In [ ]:
!cp /workspace/Kneron_yolov3_inference/model.py  /data1/keras_yolo3/yolo3/model.py 
!cp /workspace/Kneron_yolov3_inference/nef.py /workspace/E2E_Simulator/python_flow/nef/nef.py  
!cp /workspace/Kneron_yolov3_inference/kneron_inference.py /workspace/E2E_Simulator/python_flow/kneron_inference.py 

## Download COCO dataset

In [ ]:
!mkdir /workspace/COCO 
%cd /workspace/COCO

In [ ]:
!wget -c http://images.cocodataset.org/zips/val2017.zip
!unzip -q val2017.zip
!rm val2017.zip

In [ ]:
!mv val2017 images 
!mkdir val2017
!mv images val2017/images
!mkdir /workspace/COCO/val2017/predictions

# Yolov3 

In [ ]:
%cd /workspace/Kneron_yolov3_inference/

In [ ]:
!python parallel.py -h

In [ ]:
"""
This script will run untill no more predictions to be done.
We recommend to increase number of models, if you're using it to simulate Kneron 720, 
since this simulator uses only one thread.
"""

import multiprocessing
import os
from time import time
from os import listdir
from os.path import isfile, join
tic = time() 
step = 5 # images per iteration of script
models = 5 # number of models running in parallel
init = 3 # time between model's initialization
threads = 16 # number of workers for each model, works only for 520 model
path = '/workspace/COCO/val2017' # path to validation directory
predictions = path + '/' + 'predictions'
labels= path + '/' +  'images'


def missed(predictions, labels):    
    predictions =[f[:12] for f in listdir(predictions) if isfile(join(predictions, f))]    
    labels =[f[:12] for f in listdir(labels) if isfile(join(labels, f))]
    predicted = list(set(predictions) & set(labels))
    not_predicted = labels
    
    for element in predicted:
        #print(element)
        not_predicted.remove(element) 

    return not_predicted

# Check if there is all predictions
not_predicted = missed(predictions, labels)
all_processes =[]
for i in range(models):
    all_processes.append('parallel.py --path {} --model {} --step {} --init {} --conf-t 0.001 --threads {}'.format(path, i, step, init, threads))

while len(not_predicted):
    # This block of code enables us to call the script from command line.                                                                                
    def execute(process):                                                             
        os.system(f'python {process}')                                       


    process_pool = multiprocessing.Pool(processes = models)                                                        
    process_pool.map(execute, all_processes)
    
    # Check if there is all predictions
    not_predicted = missed(predictions, labels) 
    
print("Script ran for ", time() - tic)

In [ ]:
!python parallel.py --demo

# mAP evaluation

### Download COCO anotations

In [ ]:
%cd /workspace/COCO/val2017
!wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!wget https://raw.githubusercontent.com/matlab-deep-learning/Object-Detection-Using-Pretrained-YOLO-v2/main/%2Bhelper/coco-classes.txt
!unzip annotations_trainval2017.zip
!mkdir labels

In [ ]:
import json
f=open("annotations/instances_val2017.json")
coco= json.load(f)

In [ ]:
coco.keys()

In [ ]:
# COCO classes for inference
namescoco = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']

In [ ]:
# 91 classes from COCO's paper
names91 = ['empty', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat',
           'traffic light', 'fire hydrant', 'empty', 'stop sign', 'parking meter', 'bench', 'bird', 'cat',
           'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'empty', 'backpack', 
           'umbrella', 'empty', 'empty', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 
           'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 
           'bottle', 'empty', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 
           'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 
           'potted plant', 'bed', 'empty', 'dining table', 'empty', 'empty', 'toilet', 'empty', 'tv', 'laptop', 
           'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 
           'empty', 'book','clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

### Generate labels in yolo.txt format

In [ ]:
for i in range(len(coco['images'])):
    bbox = []
    for j in range(len(coco['annotations'])):
        if coco['annotations'][j]["image_id"] == coco['images'][i]["id"]: 
            x = coco['annotations'][j]["bbox"][0]
            y = coco['annotations'][j]["bbox"][1]
            w =  coco['annotations'][j]["bbox"][2]
            h = coco['annotations'][j]["bbox"][3]
            width = coco['images'][i]['width']
            height = coco['images'][i]['height']
            # Convert class id to 80 classes version
            cat = coco['annotations'][j]['category_id']
            cls = names91[cat]
            cls = namescoco.index(cls)
            bbox.append(str(cls) + " " + str((x+w/2)/width) + " " + str((y+h/2)/height)+" " + str(w/width)+" " + str(h/height)) 
            
    bbox = '\n'.join(bbox)
    with open('/workspace/COCO/val2017/labels/{}.txt'.format(coco['images'][i]["file_name"][:12]), 'w') as f:
        f.write(str(bbox))

In [ ]:
# Evaluate predictions on these labels with https://github.com/rafaelpadilla/review_object_detection_metrics
# Download all needed files that located under /workspace/COCO/val2017 and use it to calculate mAP

In [ ]:
# Your results should look like this:
"""
COCO METRICS:
AP: 0.30763665649871075
AP50: 0.5915429205728568
AP75: 0.29242076553561813
APsmall: 0.10837458746989537
APmedium: 0.29600038344642876
APlarge: 0.4424104857026416
AR1: 0.2567070573097393
AR10: 0.4102890791340148
AR100: 0.4285984240348608
ARsmall: 0.23691939502550804
ARmedium: 0.44522361970282825
ARlarge: 0.5428652114891923
"""